## Week 5 Part 2 Assignment:
## CUNY MSDS DATA620 - Web Analytics
---
### Team5: Christopher Estevez, Meaghan Burke, Rickidon Singh,  Ritesh Lohiya, Rose Koh
### 07/16/2018 (due date)
##### python version: 2.7
---

## Document Classification


It can be useful to be able to classify new "test" documents using already classified "training" documents.  A common example is using a corpus of labeled spam and ham (non-spam) e-mails to predict whether or not a new document is spam.  Here is one example of such data:  http://archive.ics.uci.edu/ml/datasets/Spambase

For this project, you can either use the above dataset to predict the class of new documents (either withheld from the training dataset or from another source such as your own spam folder).

For more adventurous students, you are welcome (encouraged!) to come up a different set of documents (including scraped web pages!?) that have already been classified (e.g. tagged), then analyze these documents to predict how new documents should be classified.

This assignment is due end of day on Monday, July 16th.  You may work in a small team if you want.


## Dataset Source and Description
The team used the dataset from http://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.names
Header names:  http://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.names
Data without headers: http://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data

The last column of 'spambase.data' denotes whether the e-mail was considered spam (1) or not (0), i.e. unsolicited commercial e-mail. Most of the attributes indicate whether a particular word or character was frequently occuring in the e-mail. The run-length attributes (55-57) measure the length of sequences of consecutive capital letters. For the statistical measures of each attribute, see the end of this file. Here are the definitions of the attributes:

48 continuous real [0,100] attributes of type word_freq_WORD
= percentage of words in the e-mail that match WORD, i.e. 100 * (number of times the WORD appears in the e-mail) / total number of words in e-mail. A "word" in this case is any string of alphanumeric characters bounded by non-alphanumeric characters or end-of-string.

6 continuous real [0,100] attributes of type char_freq_CHAR]
= percentage of characters in the e-mail that match CHAR, i.e. 100 * (number of CHAR occurences) / total characters in e-mail

1 continuous real [1,...] attribute of type capital_run_length_average
= average length of uninterrupted sequences of capital letters

1 continuous integer [1,...] attribute of type capital_run_length_longest
= length of longest uninterrupted sequence of capital letters

1 continuous integer [1,...] attribute of type capital_run_length_total
= sum of length of uninterrupted sequences of capital letters
= total number of capital letters in the e-mail

1 nominal {0,1} class attribute of type spam
= denotes whether the e-mail was considered spam (1) or not (0), i.e. unsolicited commercial e-mail. 

In [33]:
import nltk
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn import svm
import sklearn.metrics as sm
import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
names = ["word_freq_order", "word_freq_mail", "word_freq_receive", "word_freq_will",
       "word_freq_people", "word_freq_report", "word_freq_addresses",
       "word_freq_free", "word_freq_business", "word_freq_email", "word_freq_you",
       "word_freq_credit", "word_freq_your", "word_freq_font", "word_freq_000",
       "word_freq_money", "word_freq_hp", "word_freq_hpl", "word_freq_george",
       "word_freq_650", "word_freq_lab", "word_freq_labs", "word_freq_telnet",
       "word_freq_857", "word_freq_data", "word_freq_415", "word_freq_85",
       "word_freq_technology", "word_freq_1999", "word_freq_parts", "word_freq_pm",
       "word_freq_direct", "word_freq_cs", "word_freq_meeting", "word_freq_original",
       "word_freq_project", "word_freq_re", "word_freq_edu", "word_freq_table",
       "word_freq_conference", "char_freq_;", "char_freq_(", "char_freq_[",
       "char_freq_!", "char_freq_$", "char_freq_#", "capital_run_length_average",
       "capital_run_length_longest", "capital_run_length_total", "spam"
       ]


## Data Analysis
First, Let's look at our data summary statistics. Next, we want to create and test a model on this dataset.  

In [38]:
spam_data = pd.read_csv("https://raw.githubusercontent.com/silverrainb/web-analytics/master/week5/spambase.csv", names = names)
spam_data.describe()

,word_freq_order,word_freq_mail,word_freq_receive,word_freq_will,word_freq_people,word_freq_report,word_freq_addresses,word_freq_free,word_freq_business,word_freq_email,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.090067,0.239413,0.059824,0.541702,0.093930,0.058626,0.049205,0.248848,0.142586,0.184745,...,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.278616,0.644755,0.201545,0.861698,0.301036,0.335184,0.258843,0.825792,0.444055,0.531122,...,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000,0.000000
75%,0.000000,0.160000,0.000000,0.800000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,...,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000,1.000000
max,5.260000,18.180000,2.610000,9.670000,5.550000,10.000000,4.410000,20.000000,7.140000,9.090000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


We should be able to get a good result having a total of 4601 cases that are madeup of 1813 spam cases and 2788 nonspam cases. 

In [39]:
count_spam = len(spam_data[spam_data.spam==1])
count_nonspam = len(spam_data[spam_data.spam==0])
print(count_spam) 
print(count_nonspam)

1813
2788


In [40]:
#Sampling the data into train test and validation sets
sampling = np.random.choice(("train", "test", "val"), p=[0.7, 0.15, 0.15], size=len(spam_data))

In [41]:
train_set, test_set, val_set =  spam_data[sampling == "train"], spam_data[sampling == "test"],spam_data[sampling == "val"]
print(len(train_set))
print(len(test_set))
print(len(val_set))
print(len(train_set)+len(test_set)+len(val_set))

3237
713
651
4601


## k Nearest Neighbors
Let's check the accuracy using the nearest neighbor algorithm. Using spam as the outcome variable and calling it train_truth, the remaining variables will be train_predictors.

In [51]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
train_truth = train_set.iloc[:,-1:].values.ravel()
train_predictors = train_set.iloc[:,:-1]
knn.fit(train_predictors, train_truth)
knn_prediction = knn.predict(train_predictors)

In [52]:
confusion_matrix = pd.crosstab(train_truth, knn_prediction, rownames=['True'], colnames=['Predicted'], margins=True)
confusion_matrix

Predicted,0,1,All
True,,,
0,1744,218,1962
1,239,1036,1275
All,1983,1254,3237


In [54]:
from sklearn.metrics import classification_report
print "\nClassification Report: \n\n" + str(classification_report(train_truth, knn_prediction)) 
from sklearn.metrics import precision_recall_fscore_support
print "\nReport:" 
precision_recall_fscore_support(train_truth, knn_prediction)


Classification Report: 

             precision    recall  f1-score   support

          0       0.88      0.89      0.88      1962
          1       0.83      0.81      0.82      1275

avg / total       0.86      0.86      0.86      3237


Report:


(array([0.87947554, 0.8261563 ]),
 array([0.88888889, 0.81254902]),
 array([0.88415716, 0.81929616]),
 array([1962, 1275]))

In [55]:
from __future__ import division
def custom_accuracy (report):
    ppv_non_spam = report[0][0] # precision = first row -- so [0], non-spam = first col, so [0]
    ppv_spam = report[0][1]
    return ((2*ppv_non_spam) + ppv_spam)/3

In [56]:
knn_accuracy = custom_accuracy(precision_recall_fscore_support(train_truth, knn_prediction))
knn_accuracy

0.861702461352115

86% is good, but let's try to improve on it using other methods such as Random Forest and SVM. 

## Using Random Forest

In [58]:
# Build confusion matrix function
def cm_metrics(Actual, Predicted):
   cm = sm.confusion_matrix(Actual, Predicted, labels=[1, 0])
   print("True positives: %d" %cm[0,0])
   print("False positives: %d" %cm[1,0])
   print("True negatives: %d" %cm[1,1])
   print("False negatives: %d" %cm[0,1])
   print
   print(sm.classification_report(Actual, Predicted, labels=[1,0], target_names=["Spam", "Ham"]))

In [59]:
# Train the classifier using Random Forest method
train_tar = train_set['spam']
train_dat = train_set.drop(labels='spam', axis=1)

randon_forest = ensemble.RandomForestClassifier(criterion="entropy", random_state=1)
randon_forest_fit = randon_forest.fit(train_dat, train_tar)

randon_forest_train = randon_forest_fit.predict(train_dat)
cm_metrics(train_tar, randon_forest_train)

True positives: 1272
False positives: 4
True negatives: 1958
False negatives: 3

             precision    recall  f1-score   support

       Spam       1.00      1.00      1.00      1275
        Ham       1.00      1.00      1.00      1962

avg / total       1.00      1.00      1.00      3237



In [44]:
#Validation data
val_tar = val_set['spam']
val_dat = val_set.drop(labels='spam', axis=1)

randon_forest_val = randon_forest_fit.predict(val_dat)
cm_metrics(val_tar, randon_forest_val)

True positives: 224
False positives: 9
True negatives: 400
False negatives: 18

             precision    recall  f1-score   support

       Spam       0.96      0.93      0.94       242
        Ham       0.96      0.98      0.97       409

avg / total       0.96      0.96      0.96       651



In [45]:
#test data
test_tar = test_set['spam']
test_dat = test_set.drop(labels='spam', axis=1)

randon_forest_test = randon_forest_fit.predict(test_dat)
cm_metrics(test_tar, randon_forest_test)

True positives: 272
False positives: 24
True negatives: 393
False negatives: 24

             precision    recall  f1-score   support

       Spam       0.92      0.92      0.92       296
        Ham       0.94      0.94      0.94       417

avg / total       0.93      0.93      0.93       713



## Using SVM

In [46]:
#Lets try SVM
train_tar = train_set['spam']
train_dat = train_set.drop(labels='spam', axis=1)
svm = svm.SVC(random_state=1)
svm_fit = svm.fit(train_dat, train_tar)

svm_train = svm_fit.predict(train_dat)
cm_metrics(train_tar, svm_train)

True positives: 1135
False positives: 50
True negatives: 1912
False negatives: 140

             precision    recall  f1-score   support

       Spam       0.96      0.89      0.92      1275
        Ham       0.93      0.97      0.95      1962

avg / total       0.94      0.94      0.94      3237



In [47]:
#Validation data
val_tar = val_set['spam']
val_dat = val_set.drop(labels='spam', axis=1)

svm_val = svm_fit.predict(val_dat)
cm_metrics(val_tar, svm_val)

True positives: 197
False positives: 69
True negatives: 340
False negatives: 45

             precision    recall  f1-score   support

       Spam       0.74      0.81      0.78       242
        Ham       0.88      0.83      0.86       409

avg / total       0.83      0.82      0.83       651



In [48]:
#test data
test_tar = test_set['spam']

test_dat = test_set.drop(labels='spam', axis=1)

svm_test = svm_fit.predict(test_dat)
cm_metrics(test_tar, svm_test)

True positives: 232
False positives: 67
True negatives: 350
False negatives: 64

             precision    recall  f1-score   support

       Spam       0.78      0.78      0.78       296
        Ham       0.85      0.84      0.84       417

avg / total       0.82      0.82      0.82       713



## Decision Tree
Let's use Decision Tree to see the accuracy by utilizing class weights

In [61]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(class_weight = {0:2,1:1})
tree.fit(train_predictors, train_truth)

DecisionTreeClassifier(class_weight={0: 2, 1: 1}, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [62]:
tree_prediction = tree.predict(train_predictors)
pd.crosstab(train_truth, tree_prediction, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0,1962,0,1962
1,2,1273,1275
All,1964,1273,3237


In [63]:
tree_accuracy = custom_accuracy(precision_recall_fscore_support(train_truth, tree_prediction))
tree_accuracy

0.9993211133740666

We can ignore the k nearest neighbor as it isn't at par when compared to the other models used.